In [ ]:
import matplotlib.pyplot as plt
from skimage import data, color, feature, io
from skimage.transform import resize 
import numpy as np

In [ ]:

def hog_feature(image, pixel_per_cell=8):
    """
    Compute HOG feature for a given image.

    Args:
        image: an image with object that we want to detect.
        pixel_per_cell: number of pixels in each cell, an argument for hog descriptor.

    Returns:
        hogFeature: a vector of hog representation.
        hogImage: an image representation of hog provided by skimage.
    """
    hogFeature, hogImage = feature.hog(image,
                                       pixels_per_cell=(pixel_per_cell, pixel_per_cell),
                                       cells_per_block=(1, 1),
                                       block_norm='L1',
                                       visualize=True,
                                       feature_vector=True)
    return hogFeature, hogImage

In [ ]:
import matplotlib.pyplot as plt
from skimage import feature, io
from skimage.transform import resize
import numpy as np

def hog_feature(image, pixel_per_cell=8):
    """
    Compute HOG feature for a given image.

    Args:
        image: an image with object that we want to detect.
        pixel_per_cell: number of pixels in each cell, an argument for hog descriptor.

    Returns:
        hogFeature: a vector of hog representation.
        hogImage: an image representation of hog provided by skimage.
    """
    hogFeature, hogImage = feature.hog(image,
                                       pixels_per_cell=(pixel_per_cell, pixel_per_cell),
                                       cells_per_block=(3, 3),
                                       block_norm='L1',
                                       visualize=True,
                                       feature_vector=True)
    return hogFeature, hogImage

def sliding_window(image, base_score, stepSize, windowSize, pixel_per_cell=8):
    """ A sliding window that checks each different location in the image,
        and finds which location has the highest hog score. The hog score is computed
        as the dot product between the hog feature of the sliding window and the hog feature
        of the template.

    Args:
        image: an np array of size (h,w).
        base_score: hog representation of the object you want to find, an array of size (m,).
        stepSize: an int of the step size to move the window.
        windowSize: a pair of ints that is the height and width of the window.

    Returns:
        max_score: float of the highest hog score.
        maxr: int of row where the max_score is found (top-left of window).
        maxc: int of column where the max_score is found (top-left of window).
        response_map: an np array of size (h,w).
    """
    (max_score, maxr, maxc) = (0, 0, 0)
    winH, winW = windowSize
    H, W = image.shape
    response_map = np.zeros((H // stepSize + 1, W // stepSize + 1))

    for r in range(0, H - winH + 1, stepSize):
        for c in range(0, W - winW + 1, stepSize):
            window = image[r:r + winH, c:c + winW]
            hogFeature, _ = hog_feature(window, pixel_per_cell)
            score = np.dot(hogFeature, base_score)
            response_map[r // stepSize, c // stepSize] = score
            if score > max_score:
                max_score = score
                maxr, maxc = r, c

    response_map = resize(response_map, (H, W))

    return (max_score, maxr, maxc, response_map)

# Load an example grayscale image
image = io.imread('imagedata/00/01_palm/frame_00_01_0001.png', as_gray=True)

# Define the template (for example, a hand region) - here we assume you have a predefined template
# You need to load or define your own template for hand detection
template = io.imread('path_to_template_image.png', as_gray=True)
template_hog, _ = hog_feature(template)

# Parameters
stepSize = 20
windowSize = template.shape

# Perform sliding window to detect the region
max_score, maxr, maxc, response_map = sliding_window(image, template_hog, stepSize, windowSize)

# Draw a rectangle around the detected region
fig, ax = plt.subplots(1, 1, figsize=(12, 6))
ax.imshow(image, cmap=plt.cm.gray)
rect = plt.Rectangle((maxc, maxr), windowSize[1], windowSize[0], edgecolor='r', facecolor='none')
ax.add_patch(rect)
plt.show()

# Display the response map
plt.imshow(response_map, cmap='hot')
plt.title('Response Map')
plt.show()


In [14]:
import cv2
import numpy as np

# Đọc video từ camera
cap = cv2.VideoCapture(0)

# Khởi tạo kích thước cửa sổ

# Khởi tạo bộ phân đoạn nền
fgbg = cv2.createBackgroundSubtractorMOG2()

while True:
    # Đọc từng khung hình
    ret, frame = cap.read()
    if not ret:
        break
    
    # Áp dụng bộ phân đoạn nền để phát hiện vật thể đang chuyển động
    fgmask = fgbg.apply(frame)
    
    # Xử lý để loại bỏ nhiễu và cải thiện kết quả
    kernel = np.ones((5,5), np.uint8)
    fgmask = cv2.morphologyEx(fgmask, cv2.MORPH_OPEN, kernel)
    
    # Tìm các đường viền của các vật thể đã phát hiện
    contours, _ = cv2.findContours(fgmask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    
    # Vẽ đường viền cho vật thể lớn nhất
    if len(contours) > 0:
        max_contour = max(contours, key=cv2.contourArea)
        x, y, w, h = cv2.boundingRect(max_contour)
        cv2.rectangle(frame, (x, y), (x+w, y+w), (0, 255, 0), 2)
    
    # Hiển thị video kết quả
    cv2.imshow('Motion Detection', frame)


    
    # Thoát nếu nhấn phím 'q'
    if cv2.waitKey(30) & 0xFF == ord('q'):
        break

# Giải phóng tài nguyên và đóng cửa sổ
cap.release()
cv2.destroyAllWindows()


In [44]:
import cv2
import numpy as np

def detect_hand_region(image):
    # Chuyển đổi ảnh sang không gian màu HSV
    hsv = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
    """
    Trong không gian màu HSV, màu da thường có thể được xác định với Hue dao động từ khoảng 0 đến 20 độ (phụ thuộc vào nguồn tham khảo), 
    Saturation từ khoảng 30 đến 150, và Value từ khoảng 40 đến 200.
    Tuy nhiên, các giá trị này có thể thay đổi tùy thuộc vào điều kiện ánh sáng và các yếu tố khác.
    """
    
    # Xác định khoảng màu da trong không gian màu HSV
    lower_skin = np.array([0, 20, 70], dtype=np.uint8)
    upper_skin = np.array([20, 255, 255], dtype=np.uint8)
    
    # Tạo mặt nạ màu da và áp dụng vào ảnh
    mask = cv2.inRange(hsv, lower_skin, upper_skin)
    
    # Loại bỏ nhiễu và tăng cường đường biên
    mask = cv2.GaussianBlur(mask, (5, 5), 0)
    edges = cv2.Canny(mask, 50, 100)
    
    # Tìm các vùng liên thông và vẽ bounding box
    contours, _ = cv2.findContours(edges, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    # If contours square more than 5% and less than 20% of the image, draw the bounding box
    image_w = image.shape[1]
    image_h = image.shape[0]
    for contour in contours:
        x, y, w, h = cv2.boundingRect(contour)
        
        if w < 0.1 * image_w or h < 0.1 * image_h:
            continue
        cv2.rectangle(image, (x, y), (x + w, y + h), (0, 255, 0), 2)
        
    
    return image


In [45]:
# Đọc video từ camera
cap = cv2.VideoCapture(0)

while True:
    # Đọc từng khung hình
    ret, frame = cap.read()
    if not ret:
        break
    
    # Áp dụng hàm detect_hand_region để phát hiện vùng tay
    output_frame = detect_hand_region(frame)
    
    # Hiển thị video kết quả
    # output_frame = cv2.cvtColor(output_frame, cv2.COLOR_BGR2HSV)
    cv2.imshow('Hand Detection', output_frame)
    
    # Thoát nếu nhấn phím 'q'
    if cv2.waitKey(30) & 0xFF == ord('q'):
        break

# Giải phóng tài nguyên và đóng cửa sổ
cap.release()
cv2.destroyAllWindows()



In [48]:
import cv2

# Load pre-trained Haar Cascade model for hand detection
hand_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_hand.xml')

# Function to detect hands in an image
def detect_hand(image):
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    hands = hand_cascade.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=5, minSize=(30, 30))
    
    # Draw rectangles around the hands
    for (x, y, w, h) in hands:
        cv2.rectangle(image, (x, y), (x+w, y+h), (0, 255, 0), 2)
    
    return image

# Load image
image_path = 'D:\S4_SU24\CPV301\hand_gesture\test.jpg'
image = cv2.imread(image_path)

# Detect hands in the image
image_with_hands = detect_hand(image)

# Display the image with hands detected
cv2.imshow('Hands Detected', image_with_hands)
cv2.waitKey(0)
cv2.destroyAllWindows()


error: OpenCV(4.9.0) D:\a\opencv-python\opencv-python\opencv\modules\imgproc\src\color.cpp:196: error: (-215:Assertion failed) !_src.empty() in function 'cv::cvtColor'
